<a href="https://colab.research.google.com/github/Dain777666/ESAA_25-1/blob/main/Code_Practice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms
from torchvision.datasets import MNIST, CIFAR10, CIFAR100
from torch.utils.data import DataLoader

import numpy as np

In [ ]:
## data loader

path = './datasets/'

transform = transforms.Compose([transforms.ToTensor()])

train_data = MNIST(root=path,train=True,transform=transform,download=True)
test_data = MNIST(root=path,train=False,transform=transform,download=True)

batch_size = 100

train_loader = DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True,num_workers=4)
test_loader = DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False,num_workers=4)

input_shape = train_data[0][0].shape # 1, 28, 28
output_shape = len(train_data.classes) # 10

In [ ]:
## model definition

class LeNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2) # stride=1(default)
        self.pool1 = nn.AvgPool2d(kernel_size=(2,2), stride=2, padding=0) # padding=0(default)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.AvgPool2d(kernel_size=(2,2), stride=2, padding=0)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(400,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,output_shape)
        # conv는 재사용 불가(학습이 수행되기 때문에)/pool, dropout은 재사용 가능

    def forward(self,x):
      # -------Feature Extraction-------
      # print(x.shape): 100, 1, 28, 28
        hidden = F.leaky_relu(self.conv1(x))
      # print(hidden.shape): 100, 6, 28, 28
        hidden = self.pool1(hidden)
      # print(hidden.shape): 100, 6, 14, 14
        hidden = F.leaky_relu(self.conv2(hidden))
      # print(hidden.shape): 100, 16, 10, 10
        hidden = self.pool2(hidden)
      # print(hidden.shape): 100, 16, 5, 5
        hidden = self.flatten(hidden)

      # -------Classification---------
      # print(hidden.shape): 100, 400
        hidden = F.leaky_relu(self.fc1(hidden))
      # print(hidden.shape): 100, 120
        hidden = F.leaky_relu(self.fc2(hidden))
      # print(hidden.shpae): 100, 84
        output = self.fc3(hidden)
      # print(output.shape): 100, 10

        return output

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps:0")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model = LeNet().to(device) # 이 부분을 제외하고는 이전 코드와 동일
loss = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)

num_epoch = 100
train_loss_list, test_loss_list = list(), list()

for i in range(num_epoch):

    # train
    model.train()

    total_loss = 0
    count = 0

    for batch_idx, (x, y) in enumerate(train_loader):

        x, y = x.to(device), y.to(device)
        y_est = model.forward(x)
        cost = loss(y_est, y)

        total_loss == cost.item()*len(x)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pred = torch.argmax(y_est, dim=1)
        count = (pred == y).sum().item()

    acc = count/len(train_data)
    ave_loss = total_loss/len(train_data)

    train_loss_list.append(ave_loss)

    if i % 1 == 0:
        print("\nEpoch %d Train: %.3f / %.3f"%(i,ave_loss,acc))

    # eval
    model.eval()

    total_loss = 0
    count = 0

    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(test_loader):

            x, y = x.to(device), y.to(device)
            y_est = model.forward(x)
            cost = loss(y_est, y)

            total_loss == cost.item()*len(x)

            pred = torch.argmax(y_est, dim=1)
            count = (pred == y).sum().item()

    acc = count/len(train_data)
    ave_loss = total_loss/len(train_data)

    train_loss_list.append(ave_loss)

    if i % 1 == 0:
        print("Epoch %d Test: %.3f / %.3f"%(i,ave_loss,acc))


print()
num_parameter = 0
for parameter in model.parameters():
    print(parameter.shape)
    num_parameter += np.prod(parameter.size())
print(num_parameter)

# SoftmaxClassifier와 비교해보면 파라미터 개수가 엄청 차이남?

In [ ]:
# CIFAR-100 Dataset

## data loader

path = './datasets/'

transform = transforms.Compose([transforms.ToTensor(), # 0~1로 Rescaling
                                transform.Normalize(mean=[0.5071, 0.4867, 0.4408],
                                                    std=[0.2675, 0.2565, 0.2761])])

train_data = CIFAR100(root=path,train=True,transform=transform,download=True)
test_data = CIFAR100(root=path,train=False,transform=transform,download=True)

batch_size = 100

train_loader = DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True,num_workers=4)
test_loader = DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False,num_workers=4)

input_shape = train_data[0][0].shape # 3 * 32 * 32
output_shape = len(train_data.classes) # 100

In [ ]:
## model definition

class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        #
        self.fe = nn.Sequential( # 100, 3, 32, 32 -> if 3*32*32=3000 -> 500 -> 500
                    nn.Conv2d(in_channels=3, out_channels=9, kernel_size=(3,3), stride=1, padding=1), # 100, 9, 32, 32
                    nn.LeakyReLU(),
                    nn.Conv2d(in_channels=9, out_channels=18, kernel_size=(5,5), stride=1, padding=2), # 100, 18, 32, 32
                    nn.LeakyReLU(),
                    nn.AvgPool2d(kernel_size=(2,2), stride=2, padding=0), # 100, 18, 16, 16
                    nn.LeakyReLU(),
                    nn.AvgPool2d(kernel_size=(2,2), stride=2, padding=0)) # 100, 32, 4, 4

        self.flatten = nn.Flatten()

        self.fcs = nn.Sequential(
                    nn.Linear(512,256),
                    nn.LeakyReLU(),
                    nn.Linear(120,84),
                    nn.Dropout(),
                    nn.Linear(256,output_shape))

    def forward(self,x):
      hidden = self.fe(x)
      hidden = self.flatten(hidden)
      output = self.fcs(hidden)

      return output

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps:0")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model = LeNet().to(device) # 이 부분을 제외하고는 이전 코드와 동일
loss = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)

num_epoch = 100
train_loss_list, test_loss_list = list(), list()

for i in range(num_epoch):

    # train
    model.train()

    total_loss = 0
    count = 0

    for batch_idx, (x, y) in enumerate(train_loader):

        x, y = x.to(device), y.to(device)
        y_est = model.forward(x)
        cost = loss(y_est, y)

        total_loss == cost.item()*len(x)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pred = torch.argmax(y_est, dim=1)
        count = (pred == y).sum().item()

    acc = count/len(train_data)
    ave_loss = total_loss/len(train_data)

    train_loss_list.append(ave_loss)

    if i % 1 == 0:
        print("\nEpoch %d Train: %.3f / %.3f"%(i,ave_loss,acc))

    # eval
    model.eval() # dropout이 들어갔으니까??

    total_loss = 0
    count = 0

    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(test_loader):

            x, y = x.to(device), y.to(device)
            y_est = model.forward(x)
            cost = loss(y_est, y)

            total_loss == cost.item()*len(x)

            pred = torch.argmax(y_est, dim=1)
            count = (pred == y).sum().item()

    acc = count/len(train_data)
    ave_loss = total_loss/len(train_data)

    train_loss_list.append(ave_loss)

    if i % 1 == 0:
        print("Epoch %d Test: %.3f / %.3f"%(i,ave_loss,acc))


print()
num_parameter = 0
for parameter in model.parameters():
    print(parameter.shape)
    num_parameter += np.prod(parameter.size())
print(num_parameter)